In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegressionCV,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import r2_score,f1_score,accuracy_score,recall_score,precision_score,mean_absolute_error,mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from statsmodels.api import OLS
from xgboost import XGBClassifier,XGBRegressor

In [128]:
train = pd.read_csv('train_users_2.csv')

In [127]:
age_gender_data = pd.read_csv('age_gender_bkts.csv')

In [126]:
countries = pd.read_csv('countries.csv')


In [125]:
test = pd.read_csv('test_users.csv')

In [129]:
submission = pd.read_csv('sample_submission_NDF.csv')


In [ ]:
sessions = pd.read_csv('sessions.csv')

In [131]:
sessions.isnull().sum()

5

In [ ]:
sessions.groupby('user_id').agg(lambda x : x.tolist())

In [ ]:
joint_dt = pd.merge(train,sessions,how='inner',left_on='id',right_on='user_id')
pd.set_option('display.max_columns', None)
joint_dt